In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense,Layer

In [15]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense,Layer
class SelectHiddenState(Layer):
    def __init__(self, **kwargs):
        super(SelectHiddenState, self).__init__(**kwargs)

    def call(self, lstm_output, scalar_input):
        # Ensure scalar_input is an integer for indexing
        timestep_index = tf.cast(tf.squeeze(scalar_input, axis=-1), tf.int32)
        # Gather the specific hidden state for each batch
        selected_state = tf.gather(lstm_output, timestep_index, batch_dims=1, axis=1)
        return selected_state

# Example usage with your model
# lstm_output is from your BiLSTM layer
# scalar_input is your additional input
max_sentence_length = 100  # Maximum length of sentence embeddings
embedding_dim = 100        # Dimension of sentence embeddings
max_char_length = 15       # Maximum length of a word in characters
char_vocab_size = 37       # Number of unique characters
num_diacritics = 9         # Number of possible diacritics for each character, including no diacritic

# Parameters
lstm_units = 32

# Character input
char_input = Input(shape=(max_char_length, char_vocab_size))

# Inputs
sentence_input = Input(shape=(max_sentence_length, embedding_dim))
scalar_input = Input(shape=(1,), name='scalar_input')

# BiLSTM layer with return_state
bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True))
bi_lstm_output, forward_h, forward_c, backward_h, backward_c = bi_lstm(sentence_input)
print( forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)
#, forward_h, forward_c, backward_h, backward_c
# Average the forward and backward states (or choose another method to combine them)
select_state_layer = SelectHiddenState()
hidden_state_nth_timestep = select_state_layer(bi_lstm_output, scalar_input)

#hidden_state_nth_timestep = bi_lstm_output[:, scalar_input[1], :]
print(bi_lstm_output.shape)

# RNN layer with initial state from BiLSTM
rnn_cell = SimpleRNNCell(64)
rnn_layer = RNN(rnn_cell, return_sequences=True)
rnn_output = rnn_layer(char_input,initial_state=hidden_state_nth_timestep)

# Output layer
output_layer = Dense(num_diacritics, activation='softmax')(rnn_output)

# Build and compile the model
# Assuming sentence_input and scalar_input are defined as Input layers
model = Model(inputs=[sentence_input, char_input,scalar_input], outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

(None, 32) (None, 32) (None, 32) (None, 32)
(None, 100, 64)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 bidirectional_1 (Bidirecti  [(None, 100, 64),            34048     ['input_4[0][0]']             
 onal)                        (None, 32),                                                         
                              (None, 32),                                                         
                              (None, 32),                                                         
                              (None, 32)]                                                         
                                

In [16]:
# X = 
# Y = 
from preprocessing import *

trainSet = readFile('dataset/train.txt')

In [17]:
sentences_without_diacritics, diacritics = get_sentences(trainSet[400:1500])

In [18]:
print(sentences_without_diacritics)

[['ريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب'], ['قوله', 'لعدم', 'ما', 'تتعلق', 'إلخ', 'أي', 'الوصية', 'قوله', 'ما', 'مر', 'أي', 'قبيل', 'قول', 'المتن', 'لغت', 'ولو', 'اقتصر', 'على', 'أوصيت', 'له', 'بشاة', 'أو', 'أعطوه', 'شاة', 'ولا', 'غنم', 'له', 'عند', 'الموت', 'هل', 'تبطل', 'الوصية', 'أو', 'يشترى', 'له', 'شاة', 'ويؤخذ', 'من', 'قوله', 'الآتي', 'كما', 'لو', 'لم', 'يقل', 'من', 'مالي', 'ولا', 'من', 'غنمي', 'أنها', 'لا', 'تبطل'], ['وعبارة', 'الكنز', 'ولو', 'لم', 'يقل', 'من', 'مالي', 'ولا', 'من', 'غنمي', 'لم', 'يتعين', 'غنمه', 'إن', 'كانت', 'انتهت', 'ا', 'ه', 'سم', 'قوله', 'فيعطى', 'واحدة', 'منها', 'إلخ', 'كما', 'لو', 'كانت', 'موجودة', 'عند', 'الوصية', 'والموت'], ['ولا', 'يجوز', 'أن', 'يعطى', 'واحدة', 'من', 'غير', 'غنمه', 'في', 'الصورتين', 'وإن', 'تراضيا'], ['لأنه', 'صلح', 'على', 'مجهول', 'مغني', 'ونهاية', 'قال'

In [19]:
print(diacritics)

[[['ِ', '_', 'ِ'], ['_', '_', 'َّ', 'َ', '_', 'َ', 'ِ'], ['َ', '_', '_', 'ُّ', 'ُ', '_', 'ِ'], ['ِ', '_', 'َّ', 'َ', 'ِ'], ['َ', 'َ', 'ْ', 'ِ'], ['َ', 'ِ', 'َ'], ['َ', 'ِ', 'ْ', 'ٍ'], ['ُ', 'َ', 'َّ', 'ٌ'], ['َ', 'ْ', 'ُ'], ['َ', '_', 'ِ', 'ٍ'], ['َ', 'َ', 'ْ', 'َ', '_', 'ِ', 'ِ'], ['َ', 'َّ'], ['_', '_', 'َّ', '_', 'ِ', 'َ'], ['َ', '_', 'ِ', 'ٌ'], ['ِ', 'َ', '_', 'َّ', 'ِ'], ['َ', 'َ', '_', 'َ', '_'], ['َ', '_', 'َ'], ['َ', '_', 'ِ', 'ٌ'], ['ُ', 'َ'], ['َ', '_', '_', 'ِّ', 'ْ', 'ِ', '_', 'ِ'], ['_', 'َ', '_'], ['َ', 'ِ', 'َ'], ['_', '_', 'ِّ', 'ْ', 'َ'], ['ِ', 'َ', 'ْ', 'ِ', 'ِ'], ['ُ', 'ِ', 'َ'], ['َ', 'َ', 'ْ'], ['ُ', 'ْ', 'َ', 'َ', 'ْ']], [['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'َ', 'ِ'], ['َ', '_'], ['َ', 'َ', 'َ', 'َّ', 'ُ'], ['_', 'َ', 'ْ'], ['َ', 'ْ'], ['_', 'ْ', 'َ', 'ِ', 'َّ', 'ُ'], ['َ', 'ْ', 'ُ', 'ُ'], ['َ', '_'], ['َ', 'َّ'], ['َ', 'ْ'], ['ُ', 'َ', 'ْ', 'َ'], ['َ', 'ْ', 'ِ'], ['_', 'ْ', 'َ', 'ْ', 'ِ'], ['َ', 'َ', 'ْ'], ['َ', 'َ', 'ْ'], ['_', 'ْ', 'َ', 'َ', 'َ'], ['َ', 'َ', '_'],

In [20]:
word2vecmodel = makeWord2VecModel(sentences_without_diacritics)
keys = word2vecmodel.wv.key_to_index
print(keys)

{'من': 0, 'ولا': 1, 'قوله': 2, 'لم': 3, 'له': 4, 'واحدة': 5, 'الوصية': 6, 'أي': 7, 'لو': 8, 'كما': 9, 'ولو': 10, 'غنمه': 11, 'غنمي': 12, 'كانت': 13, 'ما': 14, 'تبطل': 15, 'قال': 16, 'منها': 17, 'يقل': 18, 'إلخ': 19, 'عند': 20, 'يجوز': 21, 'أن': 22, 'مالك': 23, 'قول': 24, 'شاة': 25, 'أو': 26, 'على': 27, 'مالي': 28, 'تتعلق': 29, 'لعدم': 30, 'مر': 31, 'قبيل': 32, 'المتن': 33, 'بنفسه': 34, 'يستتب': 35, 'الساحر': 36, 'النجاسة': 37, 'والسجود': 38, 'للصنم': 39, 'ونحو': 40, 'ذلك': 41, 'وسحر': 42, 'محمد': 43, 'وأصحابه': 44, 'كافر': 45, 'ولم': 46, 'بالله': 47, 'تعالى': 48, 'هو': 49, 'كالزنديق': 50, 'إذا': 51, 'عمل': 52, 'السحر': 53, 'قتل': 54, 'لغت': 55, 'كاملة': 56, 'اقتصر': 57, 'ه': 58, 'فيعطى': 59, 'موجودة': 60, 'والموت': 61, 'يعطى': 62, 'غير': 63, 'في': 64, 'الصورتين': 65, 'وإن': 66, 'تراضيا': 67, 'لأنه': 68, 'صلح': 69, 'مجهول': 70, 'مغني': 71, 'ونهاية': 72, 'ع': 73, 'سم': 74, 'ا': 75, 'أوصيت': 76, 'انتهت': 77, 'بشاة': 78, 'أعطوه': 79, 'ش': 80, 'غنم': 81, 'الموت': 82, 'هل': 83, 'يشترى': 84, 

In [21]:
def getEmbeddingsSentences(sentences, word2vecmodel):
    embeddingSentences = [] # list of all sentences
    keys = word2vecmodel.wv.key_to_index
    for s in sentences:
        embeddingTemp = []  # list for one sentence
        for w in s:
            if w in keys:
                embeddingTemp.append(word2vecmodel.wv[w])
            ### unknown OOV till now
        embeddingSentences.append(embeddingTemp)
    return embeddingSentences
embeddingsSentences = getEmbeddingsSentences(sentences_without_diacritics, word2vecmodel)

In [22]:
harakat   = {1614:1,1615:2,1616:3,1618:4,1617:5,1611:6,1612:7,1613:8, 95:9}

def get_diacritic_hot_vector(haraka):
    vector = [0 for _ in range(9)]
    print("haraka:" + haraka)
    vector[harakat[ord(haraka)] - 1] = 1
    return vector

In [23]:
def getDiacriticEncoding(wordDi):
    word_embedding = list()
    for w in wordDi:
        if (len(w) > 1):
            # print(ord(w[0]))
            print(ord(w[1]))
            w = w[0]
        word_embedding.append(get_diacritic_hot_vector(w))
    return word_embedding
# getDiacriticEncoding('ًُ')

In [54]:
X  = numpy.array([]) # input
Y  = numpy.array([]) # output
import numpy as np
for i in range(len(embeddingsSentences)):
    for j in range(len(embeddingsSentences[i])):
        x = numpy.array([])
        x=np.append(x,numpy.array([embeddingsSentences[i]]))
        x=np.append(x,[getCharacterEncoding(sentences_without_diacritics[i][j])])
        x=np.append(x,j)
        X=np.append(X,x)
        ### y ###
        y = getDiacriticEncoding(diacritics[i][j])
        Y=np.append(Y,y)
print(X.shape)

haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:_
1617
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ُ
haraka:ُ
haraka:_
haraka:ِ
haraka:ِ
haraka:_
1617
haraka:َ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ْ
haraka:ِ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ِ
haraka:ْ
haraka:ٍ
haraka:ُ
haraka:َ
1617
haraka:َ
haraka:ٌ
haraka:َ
haraka:ْ
haraka:ُ
haraka:َ
haraka:_
haraka:ِ
haraka:ٍ
haraka:َ
haraka:َ
haraka:ْ
haraka:َ
haraka:_
haraka:ِ
haraka:ِ
haraka:َ
1617
haraka:َ
haraka:_
haraka:_
1617
haraka:َ
haraka:_
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ِ
haraka:َ
haraka:_
1617
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ُ
haraka:َ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:َ
haraka:ِ
haraka:َ
haraka:ْ
haraka:ِ
haraka:ِ
h

In [66]:
import numpy as np
import tensorflow as tf

X_embeddings = []  # List to hold embedding arrays
X_scalars = []     # List to hold scalar values
Y = []             # List to hold output

for i in range(len(embeddingsSentences)):
    for j in range(len(embeddingsSentences[i])):
        # Add embeddings
        X_embeddings.append(np.array(embeddingsSentences[i]))

        # Add scalar values as a tuple or list
        char_encoding = getCharacterEncoding(sentences_without_diacritics[i][j])
        X_scalars.append([char_encoding, j])

        # For Y
        y = getDiacriticEncoding(diacritics[i][j])
        Y.append(y)

# Convert to tensors
X_embeddings_tensor = tf.ragged.constant(X_embeddings)  # Ragged tensor for embeddings
X_scalars_tensor = tf.convert_to_tensor(X_scalars)      # Regular tensor for scalars
Y_tensor = tf.convert_to_tensor(Y)

# Check the shapes
print(X_embeddings_tensor.shape)
print(X_scalars_tensor.shape)
print(Y_tensor.shape)


haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:_
1617
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ُ
haraka:ُ
haraka:_
haraka:ِ
haraka:ِ
haraka:_
1617
haraka:َ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ْ
haraka:ِ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ِ
haraka:ْ
haraka:ٍ
haraka:ُ
haraka:َ
1617
haraka:َ
haraka:ٌ
haraka:َ
haraka:ْ
haraka:ُ
haraka:َ
haraka:_
haraka:ِ
haraka:ٍ
haraka:َ
haraka:َ
haraka:ْ
haraka:َ
haraka:_
haraka:ِ
haraka:ِ
haraka:َ
1617
haraka:َ
haraka:_
haraka:_
1617
haraka:َ
haraka:_
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ِ
haraka:َ
haraka:_
1617
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ُ
haraka:َ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:َ
haraka:ِ
haraka:َ
haraka:ْ
haraka:ِ
haraka:ِ
h

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [67]:
X  = list() # input
Y  = list() # output
for i in range(len(embeddingsSentences)):
    for j in range(len(embeddingsSentences[i])):
        x = list()
        x.append(embeddingsSentences[i])
        x.append(getCharacterEncoding(sentences_without_diacritics[i][j]))
        x.append(j)
        X.append(x)
        ### y ###
        y = getDiacriticEncoding(diacritics[i][j])
        Y.append(y)
    
print(X[0][1])

haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:_
1617
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ُ
haraka:ُ
haraka:_
haraka:ِ
haraka:ِ
haraka:_
1617
haraka:َ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ْ
haraka:ِ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ِ
haraka:ْ
haraka:ٍ
haraka:ُ
haraka:َ
1617
haraka:َ
haraka:ٌ
haraka:َ
haraka:ْ
haraka:ُ
haraka:َ
haraka:_
haraka:ِ
haraka:ٍ
haraka:َ
haraka:َ
haraka:ْ
haraka:َ
haraka:_
haraka:ِ
haraka:ِ
haraka:َ
1617
haraka:َ
haraka:_
haraka:_
1617
haraka:َ
haraka:_
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ِ
haraka:َ
haraka:_
1617
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ُ
haraka:َ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:َ
haraka:ِ
haraka:َ
haraka:ْ
haraka:ِ
haraka:ِ
h

In [71]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_sentence = []  # List for sentence_input
X_char = []      # List for char_input
X_scalar = []    # List for scalar_input
Y = []           # List for output

for i in range(len(embeddingsSentences)):
    for j in range(len(embeddingsSentences[i])):
        # Add embeddings
        X_sentence.append(embeddingsSentences[i])

        # Prepare char_input
        char_encoding = getCharacterEncoding(sentences_without_diacritics[i][j])
        # Ensure char_encoding is shaped as (max_char_length, char_vocab_size)
        # This might require reshaping or padding depending on your getCharacterEncoding function
        X_char.append(char_encoding)

        # Scalar input
        X_scalar.append([j])

        # Output
        y = getDiacriticEncoding(diacritics[i][j])
        Y.append(y)

# Padding X_sentence
# Assuming max_sentence_length and embedding_dim are defined
X_sentence_padded = pad_sequences(X_sentence, maxlen=max_sentence_length, padding='post', dtype='float32')
X_character_padded = pad_sequences(X_char, maxlen=max_char_length, padding='post', dtype='float32')
Y__padded = pad_sequences(Y, maxlen=max_char_length, padding='post', dtype='float32')

# Convert lists to numpy arrays
X_sentence = np.array(X_sentence_padded)
X_char = np.array(X_character_padded)
X_scalar = np.array(X_scalar)
X_final
Y = np.array(Y__padded)

print(X_sentence.shape, X_char.shape, X_scalar.shape, Y.shape)


haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:_
1617
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ُ
haraka:ُ
haraka:_
haraka:ِ
haraka:ِ
haraka:_
1617
haraka:َ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ْ
haraka:ِ
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:ِ
haraka:ْ
haraka:ٍ
haraka:ُ
haraka:َ
1617
haraka:َ
haraka:ٌ
haraka:َ
haraka:ْ
haraka:ُ
haraka:َ
haraka:_
haraka:ِ
haraka:ٍ
haraka:َ
haraka:َ
haraka:ْ
haraka:َ
haraka:_
haraka:ِ
haraka:ِ
haraka:َ
1617
haraka:َ
haraka:_
haraka:_
1617
haraka:َ
haraka:_
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ِ
haraka:َ
haraka:_
1617
haraka:َ
haraka:ِ
haraka:َ
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:َ
haraka:_
haraka:ِ
haraka:ٌ
haraka:ُ
haraka:َ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:ِ
haraka:_
haraka:ِ
haraka:_
haraka:َ
haraka:_
haraka:َ
haraka:ِ
haraka:َ
haraka:_
haraka:_
1617
haraka:ِ
haraka:ْ
haraka:َ
haraka:ِ
haraka:َ
haraka:ْ
haraka:ِ
haraka:ِ
h

In [57]:
print(x)

[[array([-8.6194072e-03,  3.6662782e-03,  5.1877904e-03,  5.7404060e-03,
        7.4694911e-03, -6.1716656e-03,  1.1047125e-03,  6.0493494e-03,
       -2.8413848e-03, -6.1743795e-03, -4.1114856e-04, -8.3720060e-03,
       -5.6005633e-03,  7.1064476e-03,  3.3553925e-03,  7.2244219e-03,
        6.7994562e-03,  7.5301109e-03, -3.7890249e-03, -5.6519831e-04,
        2.3484014e-03, -4.5158374e-03,  8.3882548e-03, -9.8603358e-03,
        6.7657670e-03,  2.9121095e-03, -4.9342490e-03,  4.3973173e-03,
       -1.7399262e-03,  6.7108874e-03,  9.9654859e-03, -4.3622348e-03,
       -5.9633976e-04, -5.6975926e-03,  3.8477452e-03,  2.7883479e-03,
        6.8911030e-03,  6.1023729e-03,  9.5375814e-03,  9.2732897e-03,
        7.9012951e-03, -6.9908272e-03, -9.1558043e-03, -3.5499129e-04,
       -3.0983118e-03,  7.8949267e-03,  5.9358068e-03, -1.5467355e-03,
        1.5131170e-03,  1.7902276e-03,  7.8178952e-03, -9.5113264e-03,
       -2.0500859e-04,  3.4705850e-03, -9.3895441e-04,  8.3821947e-03,
    

In [62]:
#model.fit({X}, Y, epochs= 30)
# print(X)
# print("yyyyyyyyy")
print(Y)

[[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0]], [[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [63]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense, Layer, ZeroPadding1D

class SelectHiddenState(Layer):
    def __init__(self, **kwargs):
        super(SelectHiddenState, self).__init__(**kwargs)

    def call(self, lstm_output, scalar_input):
        timestep_index = tf.cast(tf.squeeze(scalar_input, axis=-1), tf.int32)
        selected_state = tf.gather(lstm_output, timestep_index, batch_dims=1, axis=1)
        return selected_state

# Parameters
max_sentence_length = 100
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 8
lstm_units = 32

# Inputs
char_input = Input(shape=(max_char_length, char_vocab_size))
sentence_input = Input(shape=(max_sentence_length, embedding_dim))
scalar_input = Input(shape=(1,), name='scalar_input')

# Padding layer for sentence_input (adjust padding as needed)
sentence_padding_layer = ZeroPadding1D(padding=(1, 1))  # Example padding
padded_sentence_input = sentence_padding_layer(sentence_input)
padding_layer = ZeroPadding1D(padding=(1, 1))  # Example padding
padded_char_input = padding_layer(char_input)
# BiLSTM layer
bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True))
bi_lstm_output, forward_h, forward_c, backward_h, backward_c = bi_lstm(padded_sentence_input)

# Select state layer
select_state_layer = SelectHiddenState()
hidden_state_nth_timestep = select_state_layer(bi_lstm_output, scalar_input)

# RNN layer
rnn_cell = SimpleRNNCell(64)
rnn_layer = RNN(rnn_cell, return_sequences=True)
rnn_output = rnn_layer(padded_char_input, initial_state=hidden_state_nth_timestep)

# Output layer
output_layer = Dense(num_diacritics, activation='softmax')(rnn_output)

# Model
model = Model(inputs=[sentence_input, char_input, scalar_input], outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 100, 100)]           0         []                            
                                                                                                  
 zero_padding1d_8 (ZeroPadd  (None, 102, 100)             0         ['input_14[0][0]']            
 ing1D)                                                                                           
                                                                                                  
 input_13 (InputLayer)       [(None, 15, 36)]             0         []                            
                                                                                                  
 bidirectional_6 (Bidirecti  [(None, 102, 64),            34048     ['zero_padding1d_8[0][0]

In [65]:
model.fit(X, Y, epochs= 30)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>", \'(<class \\\'list\\\'> containing values of types {"<class \\\'numpy.ndarray\\\'>"})\', \'(<class \\\'list\\\'> containing values of types {\\\'(<class \\\\\\\'list\\\\\\\'> containing values of types {"<class \\\\\\\'int\\\\\\\'>"})\\\'})\'})'}), (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {\'(<class \\\'list\\\'> containing values of types {"<class \\\'int\\\'>"})\'})'})